# 02 — Build bridge ontology (PPMI ↔ PDON/PMDO)

This notebook creates a non-destructive **bridge ontology** that:

- imports PDON and PMDO (as-is)
- defines a minimal T-box for Subject/Visit/Observation
- records the PPMI→ontology mapping from `mapping/ppmi_pdon_pmdo_mapping.csv`
- serialises the bridge ontology to `ontologies/ppmi_bridge.ttl`

> Assumption: you have `ontologies/pdon.xrdf` and `ontologies/pmdo.xrdf` available in the repo.


In [ ]:
!pip -q install rdflib pandas owlrl


In [ ]:
import os
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, OWL, XSD

REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), "ppmi-ontology-alignment")) if os.path.isdir("ppmi-ontology-alignment") else os.getcwd()
ONT_DIR = os.path.join(REPO_ROOT, "ontologies")
MAP_PATH = os.path.join(REPO_ROOT, "mapping", "ppmi_pdon_pmdo_mapping.csv")
OUT_PATH = os.path.join(ONT_DIR, "ppmi_bridge.ttl")

PDON_PATH = os.path.join(ONT_DIR, "pdon.xrdf")
PMDO_PATH = os.path.join(ONT_DIR, "pmdo.xrdf")

print("REPO_ROOT:", REPO_ROOT)
print("Exists PDON:", os.path.exists(PDON_PATH), PDON_PATH)
print("Exists PMDO:", os.path.exists(PMDO_PATH), PMDO_PATH)
print("Exists mapping:", os.path.exists(MAP_PATH), MAP_PATH)


In [ ]:
mapping = pd.read_csv(MAP_PATH)
mapping.head(10)


In [ ]:
pdon_g = Graph()
pmdo_g = Graph()

pdon_g.parse(PDON_PATH)
pmdo_g.parse(PMDO_PATH)

print("PDON triples:", len(pdon_g))
print("PMDO triples:", len(pmdo_g))


In [ ]:
def get_ontology_iris(g: Graph):
    return sorted({str(s) for s in g.subjects(RDF.type, OWL.Ontology)})

pdon_iris = get_ontology_iris(pdon_g)
pmdo_iris = get_ontology_iris(pmdo_g)

print("PDON ontology IRI(s):", pdon_iris)
print("PMDO ontology IRI(s):", pmdo_iris)


In [ ]:
bridge = Graph()

PPMI = Namespace("http://example.org/ppmi-ontology-alignment#")  # replace later
bridge.bind("ppmi", PPMI)
bridge.bind("owl", OWL)
bridge.bind("rdfs", RDFS)
bridge.bind("rdf", RDF)
bridge.bind("xsd", XSD)

BRIDGE_IRI = URIRef("http://example.org/ppmi-ontology-alignment")  # replace later
bridge.add((BRIDGE_IRI, RDF.type, OWL.Ontology))
bridge.add((BRIDGE_IRI, RDFS.label, Literal("PPMI–PDON/PMDO bridge ontology")))
bridge.add((BRIDGE_IRI, RDFS.comment, Literal("Non-destructive bridge ontology for mapping PPMI longitudinal variables to PDON and PMDO concepts.")))

# owl:imports
bridge.add((BRIDGE_IRI, OWL.imports, URIRef(pdon_iris[0] if pdon_iris else "http://www.semanticweb.org/ontologies/2011/1/Ontology1296772722296.owl")))
bridge.add((BRIDGE_IRI, OWL.imports, URIRef(pmdo_iris[0] if pmdo_iris else "http://www.case.edu/PMDO")))


In [ ]:
# Minimal T-box

for cls,label in [
    ("Subject","Subject (PPMI participant)"),
    ("Visit","Visit (PPMI timepoint)"),
    ("Observation","Observation / measurement at a visit"),
    ("DiagnosisObservation","Diagnosis observation (coded)"),
    ("ImagingObservation","Imaging-derived observation"),
]:
    c = PPMI[cls]
    bridge.add((c, RDF.type, OWL.Class))
    bridge.add((c, RDFS.label, Literal(label)))

def add_objprop(local, domain, range_, label):
    p = PPMI[local]
    bridge.add((p, RDF.type, OWL.ObjectProperty))
    bridge.add((p, RDFS.domain, PPMI[domain]))
    bridge.add((p, RDFS.range, range_))
    bridge.add((p, RDFS.label, Literal(label)))
    return p

add_objprop("hasVisit","Subject",PPMI["Visit"],"has visit")
add_objprop("hasObservation","Visit",PPMI["Observation"],"has observation")
add_objprop("observesConcept","Observation",OWL.Thing,"observes concept (PDON/PMDO/MDO class)")
add_objprop("refersToPdonConcept","DiagnosisObservation",OWL.Thing,"refers to PDON concept")
add_objprop("relatesToRegion","ImagingObservation",OWL.Thing,"relates to anatomical region (e.g., MDO)")

def add_dataprop(local, domain, range_, label):
    p = PPMI[local]
    bridge.add((p, RDF.type, OWL.DatatypeProperty))
    bridge.add((p, RDFS.domain, PPMI[domain]))
    bridge.add((p, RDFS.range, range_))
    bridge.add((p, RDFS.label, Literal(label)))
    return p

add_dataprop("hasValue","Observation",XSD.string,"has value (literal)")
add_dataprop("visitDate","Visit",XSD.string,"visit date (raw)")
add_dataprop("visitYear","Visit",XSD.integer,"visit year index")
add_dataprop("ageAtVisit","Visit",XSD.decimal,"age at visit")


In [ ]:
# Embed mapping as annotations
mapsVariable = PPMI["mapsVariable"]
bridge.add((mapsVariable, RDF.type, OWL.AnnotationProperty))
bridge.add((mapsVariable, RDFS.label, Literal("maps PPMI variable to concept IRI")))

ppmiVariable = PPMI["ppmiVariable"]
bridge.add((ppmiVariable, RDF.type, OWL.AnnotationProperty))
bridge.add((ppmiVariable, RDFS.label, Literal("PPMI variable name")))

mappingBucket = PPMI["mappingBucket"]
bridge.add((mappingBucket, RDF.type, OWL.AnnotationProperty))
bridge.add((mappingBucket, RDFS.label, Literal("mapping bucket")))

mappingConfidence = PPMI["mappingConfidence"]
bridge.add((mappingConfidence, RDF.type, OWL.AnnotationProperty))
bridge.add((mappingConfidence, RDFS.label, Literal("mapping confidence")))

for _,r in mapping.iterrows():
    var = str(r["Variable"]).strip()
    target = str(r["TargetIRI"]).strip()
    bucket = str(r["MappingBucket"]).strip()
    conf = str(r["Confidence"]).strip()

    node = URIRef(f"{PPMI}mapping/{var}")
    bridge.add((node, ppmiVariable, Literal(var)))
    if bucket and bucket != "nan":
        bridge.add((node, mappingBucket, Literal(bucket)))
    if conf and conf != "nan":
        bridge.add((node, mappingConfidence, Literal(conf)))
    if target and target != "nan" and target != "":
        bridge.add((node, mapsVariable, URIRef(target)))

print("Bridge triples:", len(bridge))


In [ ]:
bridge.serialize(destination=OUT_PATH, format="turtle")
print("Wrote:", OUT_PATH)
